# Model Selection

### Get API Key

In [1]:
from helper import load_mistral_api_key
api_key, dlai_endpoint = load_mistral_api_key(ret_key=True)

- Note: in the classroom, if you print out this `api_key` variable, it is not a real API key (for security reasons).
- If you wish to run this code on your own machine, outside of the classroom, you can still reuse the code that you see in `helper.py`.
- It uses [python-dotenv](https://pypi.org/project/python-dotenv/) library to securely save and load sensitive information such as API keys.

In [2]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

def mistral(user_message, model="mistral-small-latest", is_json=False):
    client = MistralClient(api_key=api_key, endpoint=dlai_endpoint)
    messages = [ChatMessage(role="user", content=user_message)]

    if is_json:
        chat_response = client.chat(
            model=model, messages=messages, response_format={"type": "json_object"}
        )
    else:
        chat_response = client.chat(model=model, messages=messages)

    return chat_response.choices[0].message.content

## Mistral Small

Good for simple tasks, fast inference, lower cost.
- classification

In [3]:
prompt = """
Classify the following email to determine if it is spam or not.
Only respond with the exact text "Spam" or "Not Spam". 

# Email:
🎉 Urgent! You've Won a $1,000,000 Cash Prize! 
💰 To claim your prize, please click on the link below: 
https://bit.ly/claim-your-prize
"""

In [4]:
mistral(prompt, model="mistral-small-latest")

'Spam'

## Mistral Medium

Good for intermediate tasks such as language transformation.
- Composing text based on provided context (e.g. writing a customer service email based on purchase information).

In [5]:
prompt = """
Compose a welcome email for new customers who have just made 
their first purchase with your product. 
Start by expressing your gratitude for their business, 
and then convey your excitement for having them as a customer. 
Include relevant details about their recent order. 
Sign the email with "The Fun Shop Team".

Order details:
- Customer name: Anna
- Product: hat 
- Estimate date of delivery: Feb. 25, 2024
- Return policy: 30 days
"""

In [6]:
response_medium = mistral(prompt, model="mistral-medium-latest")

In [7]:
print(response_medium)

**Subject:** Welcome to The Fun Shop, Anna! Your Hat is on the Way 🎉

**Hi Anna,**

We’re so grateful you chose **The Fun Shop** for your first purchase—thank you for supporting our small business! 🎩✨ Your new hat is already packed with care and will be on its way to you soon.

**Here’s a quick recap of your order:**
- **Product:** A stylish hat (we know you’ll love it!)
- **Estimated Delivery:** February 25, 2024
- **Return Policy:** 30 days—because we want you to be 100% happy!

We’re thrilled to have you as part of our community and can’t wait to see how you style your new hat. If you have any questions or just want to share a photo of your look, reply to this email—we’d love to hear from you!

Welcome to the fun side of shopping. 😊

**The Fun Shop Team**


## Mistral Large: 

Good for complex tasks that require advanced reasoning.
- Math and reasoning with numbers.

In [8]:
prompt = """
Calculate the difference in payment dates between the two \
customers whose payment amounts are closest to each other \
in the following dataset. Do not write code.

# dataset: 
'{
  "transaction_id":{"0":"T1001","1":"T1002","2":"T1003","3":"T1004","4":"T1005"},
    "customer_id":{"0":"C001","1":"C002","2":"C003","3":"C002","4":"C001"},
    "payment_amount":{"0":125.5,"1":89.99,"2":120.0,"3":54.3,"4":210.2},
"payment_date":{"0":"2021-10-05","1":"2021-10-06","2":"2021-10-07","3":"2021-10-05","4":"2021-10-08"},
    "payment_status":{"0":"Paid","1":"Unpaid","2":"Paid","3":"Paid","4":"Pending"}
}'
"""

In [9]:
response_small = mistral(prompt, model="mistral-small-latest")

In [10]:
print(response_small)

To solve this problem, we need to find the two customers whose payment amounts are closest to each other and then calculate the difference in their payment dates. Here's the step-by-step reasoning:

1. **Extract Payment Amounts and Dates**:
   - The payment amounts are: 125.5, 89.99, 120.0, 54.3, 210.2.
   - The corresponding payment dates are: "2021-10-05", "2021-10-06", "2021-10-07", "2021-10-05", "2021-10-08".

2. **Identify the Two Closest Payment Amounts**:
   - Sort the payment amounts in ascending order: 54.3, 89.99, 120.0, 125.5, 210.2.
   - The two closest amounts are 120.0 and 125.5 (difference of 5.5).

3. **Find the Corresponding Payment Dates**:
   - The payment amount 120.0 corresponds to the date "2021-10-07".
   - The payment amount 125.5 corresponds to the date "2021-10-05".

4. **Calculate the Difference in Payment Dates**:
   - Convert the dates to a comparable format (e.g., days since a reference date or datetime objects).
   - "2021-10-05" is 2 days before "2021-10

In [11]:
response_large = mistral(prompt, model="mistral-large-latest")

In [12]:
print(response_large)

To find the difference in payment dates between the two customers whose payment amounts are closest to each other, follow these steps:

1. **Extract payment amounts and dates**:
   - T1001: $125.50 (2021-10-05)
   - T1002: $89.99 (2021-10-06)
   - T1003: $120.00 (2021-10-07)
   - T1004: $54.30 (2021-10-05)
   - T1005: $210.20 (2021-10-08)

2. **Calculate the absolute differences between all pairs of payment amounts**:
   - |125.50 - 89.99| = 35.51
   - |125.50 - 120.00| = 5.50
   - |125.50 - 54.30| = 71.20
   - |125.50 - 210.20| = 84.70
   - |89.99 - 120.00| = 30.01
   - |89.99 - 54.30| = 35.69
   - |89.99 - 210.20| = 120.21
   - |120.00 - 54.30| = 65.70
   - |120.00 - 210.20| = 90.20
   - |54.30 - 210.20| = 155.90

3. **Identify the smallest difference**:
   - The smallest difference is **5.50** between **$125.50 (T1001)** and **$120.00 (T1003)**.

4. **Extract the payment dates for these two transactions**:
   - T1001: 2021-10-05
   - T1003: 2021-10-07

5. **Calculate the difference 

## Expense reporting task

In [13]:
transactions = """
McDonald's: 8.40
Safeway: 10.30
Carrefour: 15.00
Toys R Us: 20.50
Panda Express: 10.20
Beanie Baby Outlet: 25.60
World Food Wraps: 22.70
Stuffed Animals Shop: 45.10
Sanrio Store: 85.70
"""

prompt = f"""
Given the purchase details, how much did I spend on each category:
1) restaurants
2) groceries
3) stuffed animals and props
{transactions}
"""

In [14]:
response_small = mistral(prompt, model="mistral-small-latest")
print(response_small)

Here's the breakdown of your spending by category:

1) **Restaurants**:
   - McDonald's: $8.40
   - Panda Express: $10.20
   - **Total**: $8.40 + $10.20 = **$18.60**

2) **Groceries**:
   - Safeway: $10.30
   - Carrefour: $15.00
   - World Food Wraps: $22.70
   - **Total**: $10.30 + $15.00 + $22.70 = **$48.00**

3) **Stuffed Animals and Props**:
   - Toys R Us: $20.50
   - Beanie Baby Outlet: $25.60
   - Stuffed Animals Shop: $45.10
   - Sanrio Store: $85.70
   - **Total**: $20.50 + $25.60 + $45.10 + $85.70 = **$176.90**

### Summary:
- **Restaurants**: $18.60
- **Groceries**: $48.00
- **Stuffed Animals and Props**: $176.90


In [15]:
response_large = mistral(prompt, model="mistral-large-latest")
print(response_large)

Here's the breakdown of your spending by category:

1) **Restaurants**:
   - McDonald's: $8.40
   - Panda Express: $10.20
   - World Food Wraps: $22.70
   **Total: $41.30**

2) **Groceries**:
   - Safeway: $10.30
   - Carrefour: $15.00
   **Total: $25.30**

3) **Stuffed Animals and Props**:
   - Toys R Us: $20.50
   - Beanie Baby Outlet: $25.60
   - Stuffed Animals Shop: $45.10
   - Sanrio Store: $85.70
   **Total: $176.90**


## Writing and checking code

In [16]:
user_message = """
Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.

You may assume that each input would have exactly one solution, and you may not use the same element twice.

You can return the answer in any order.

Your code should pass these tests:

assert twoSum([2,7,11,15], 9) == [0,1]
assert twoSum([3,2,4], 6) == [1,2]
assert twoSum([3,3], 6) == [0,1]
"""

In [17]:
print(mistral(user_message, model="mistral-large-latest"))

To solve this problem, we need to find two indices in an array such that the corresponding numbers add up to a given target. The solution should efficiently check pairs of numbers without using the same element twice and return the indices of the valid pair.

### Approach
1. **Brute Force Method**: The simplest approach is to check every possible pair of numbers in the array to see if they sum up to the target. This involves nested loops where the outer loop runs from the first element to the second last element, and the inner loop runs from the current element of the outer loop to the end of the array. The time complexity of this approach is O(n^2), which is inefficient for large arrays.

2. **Hash Map (Dictionary) Method**: A more efficient approach involves using a hash map to store the elements of the array as we iterate through them. For each element, we check if the complement (target - current element) exists in the hash map. If it does, we return the indices of the current elem

### Try out the code that the model provided
- Copy the code that the model provided and try running it!

Here is the code that was output at the time of filming:
```Python
def twoSum(nums, target):
    seen = {}
    for i, num in enumerate(nums):
        complement = target - num
        if complement in seen:
            return [seen[complement], i]
        seen[num] = i
```
- Also try running the assert statements in the original prompt
```Python
assert twoSum([2,7,11,15], 9) == [0,1]
assert twoSum([3,2,4], 6) == [1,2]
assert twoSum([3,3], 6) == [0,1]
```

In [19]:
def twoSum(nums, target):
    num_to_index = {}
    for index, num in enumerate(nums):
        complement = target - num
        if complement in num_to_index:
            return [num_to_index[complement], index]
        num_to_index[num] = index
    return []

In [20]:
assert twoSum([2,7,11,15], 9) == [0,1]

In [21]:
assert twoSum([3,2,4], 6) == [1,2]

In [22]:
assert twoSum([3,3], 6) == [0,1]

## Natively Fluent in English, French, Spanish, German, and Italian
- This means that you can use Mistral models for more than translating from one language to another.
- If you are a native Spanish speaker, for instance, you can communicate with Mistral models in Spanish for any of your tasks.

In [23]:
user_message = """
Lequel est le plus lourd une livre de fer ou un kilogramme de plume
"""

In [24]:
print(mistral(user_message, model="mistral-large-latest"))

Pour comparer le poids d'une livre de fer et d'un kilogramme de plumes, il faut d'abord convertir les unités dans le même système.

- **1 livre (lb)** ≈ **0,4536 kilogrammes (kg)**
- **1 kilogramme (kg)** = **1 kg**

Donc :
- **1 livre de fer** ≈ **0,4536 kg**
- **1 kg de plumes** = **1 kg**

**Conclusion** : **Un kilogramme de plumes est plus lourd qu'une livre de fer** (1 kg > 0,4536 kg).

*(Remarque : Si on parlait d'une "livre" dans le sens ancien français équivalent à 500 g, la réponse changerait, mais aujourd'hui, la livre internationale est définie comme 0,4536 kg.)*


### Try it out for yourself
- Try communicating with the Mistral Large model in Spanish
  - (If you need help, you can first translate a prompt from English to Spanish, and then prompt the model in Spanish).

## List of Mistral models that you can call:

You can also call the two open source mistral models via API calls.
Here is the list of models that you can try:
```
open-mistral-7b
open-mixtral-8x7b
open-mixtral-8x22b
mistral-small-latest
mistral-medium-latest
mistral-large-latest
```

For example:
```Python
mistral(prompt, model="open-mixtral-8x22b")
```

Note that we just released the `open-mixtral-8x22b` model. Check out our [release blog](https://mistral.ai/news/mixtral-8x22b/) for details. 

In [28]:
mistral("Determine the sentiment of this sentence in three category (Positive, Negative, Neutral) with the percentage: DeepLearning.AI is the best source to advance in AI.", model="open-mixtral-8x22b")

'The sentiment of the sentence "DeepLearning.AI is the best source to advance in AI" is Positive with a high percentage. The sentence expresses a clear and strong opinion that DeepLearning.AI is the best source for advancing in AI, which is a positive sentiment. There is no indication of negativity or neutrality. Therefore, I would rate it as 100% Positive, 0% Negative, and 0% Neutral.'